In [1]:
import os

import pandas as pd
from geopandas import gpd

# Package in this repository containing some common functionality
from hscap.geo import sum_per_region

In [2]:
ccm_facility_data_gpd = gpd.read_file('../data/ccm_data_by_facility.geojson')
ccm_facility_data_gpd

,Estimated Beds Available (Conventional),Estimated Additional Ventilators Required (Conventional),Physicians Required to Staff Beds (Conventional) - Lower,Physicians Required to Staff Beds (Conventional) - Upper,Respiratory Therapists Required to Staff Beds (Conventional) - Lower,Respiratory Therapists Required to Staff Beds (Conventional) - Upper,Critical Care Nurses Required to Staff Beds (Conventional) - Lower,Critical Care Nurses Required to Staff Beds (Conventional) - Upper,Estimated Beds Available (Contingency),Estimated Additional Ventilators Required (Contingency),...,Zipcode,County,DH_ID,HCRIS_ID,Total Staffed ICU Beds,Total Staffed Beds,Total Licensed Beds,Bed Occupancy Rate,ICU Bed Occupancy Rate,geometry
0,0,0,1,1,1,1,1,1,0,0,...,37660,Sullivan,6533,None,0.0,72.0,72.0,0.100000,0.100000,POINT (-82.51248 36.54377)
1,10,0,1,1,1,1,3,3,31,0,...,37660,Sullivan,3538,440176,14.0,148.0,239.0,0.321140,0.299022,POINT (-82.51436 36.55092)
2,0,0,1,1,1,1,1,1,0,0,...,46140,Hancock,6528,None,0.0,12.0,12.0,0.100000,0.100000,POINT (-85.76760 39.79502)
3,11,0,1,1,1,1,3,3,35,0,...,46140,Hancock,1226,150037,24.0,61.0,61.0,0.368291,0.543265,POINT (-85.76771 39.79490)
4,0,0,1,1,1,1,1,1,0,0,...,43210,Franklin,6523,None,0.0,87.0,87.0,0.100000,0.100000,POINT (-83.01503 39.99505)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6341,0,0,1,1,1,1,1,1,0,0,...,33034,Miami-Dade,5994,None,0.0,238.0,249.0,0.100000,0.100000,POINT (-80.49157 25.41949)
6342,0,0,1,1,1,1,1,1,0,0,...,85739,Pinal,5995,None,0.0,139.0,139.0,0.100000,0.100000,POINT (-110.88848 32.51704)
6343,0,0,1,1,1,1,1,1,0,0,...,95678,Placer,5996,None,0.0,16.0,16.0,0.100000,0.100000,POINT (-121.28634 38.73032)
6344,0,0,1,1,1,1,1,1,0,0,...,77035,Harris,5998,None,0.0,120.0,120.0,0.100000,0.100000,POINT (-95.45740 29.64700)


In [3]:
# Read in county-level data
county_gdf = gpd.read_file('../data/us_counties_with_pop.geojson')

# Filter by counties of interest
counties_of_interest = ['Alamedac County',
                        'Marin County', 
                        'Sonoma County', 
                        'Napa County', 
                        'Solano County', 
                        'Contra Costa County',
                        'Santa Cruz County']

filtered_county_gdf = county_gdf
filtered_county_gdf['NAME'] = filtered_county_gdf['NAME'] + ' County'
filtered_county_gdf = filtered_county_gdf[filtered_county_gdf['NAME'].isin(counties_of_interest)]
filtered_county_gdf = filtered_county_gdf[['NAME', 'Population', 'geometry']]
filtered_county_gdf

,NAME,Population,geometry
27,Napa County,139417.0,"POLYGON ((-122.10328 38.51335, -122.08884 38.3..."
735,Santa Cruz County,274255.0,"POLYGON ((-122.01597 37.16566, -121.75760 37.0..."
736,Solano County,446610.0,"POLYGON ((-122.08884 38.38415, -122.10328 38.5..."
741,Contra Costa County,1150215.0,"POLYGON ((-121.91014 37.73046, -121.96077 37.7..."
866,Santa Cruz County,46511.0,"POLYGON ((-111.16399 31.61869, -111.16418 31.7..."
2002,Marin County,259666.0,"POLYGON ((-122.49073 38.10975, -122.49128 38.1..."
2311,Sonoma County,499942.0,"POLYGON ((-123.26871 38.80837, -123.13625 38.8..."


In [4]:
# Get san francisco GeoJSON
if not os.path.isdir('download_data'):
    os.makedirs('download_data')
if not os.path.exists('download_data/sf-neighborhoods.geojson'):
    !wget -O download_data/sf-neighborhoods.geojson https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/san-francisco.geojson
        
sf_neighborhoods_gdf = gpd.read_file('download_data/sf-neighborhoods.geojson')


In [5]:
sf_neighborhoods_gdf['region_id'] = 1
sf_gdf = sf_neighborhoods_gdf.dissolve(by='region_id')[['geometry']]

# Source: Googling 'SF Population' (2017 Census data)
sf_gdf['NAME'] = 'San Francisco'
sf_gdf['Population'] = 884363

In [6]:
sf_gdf

,geometry,NAME,Population
region_id,,,
1,"MULTIPOLYGON (((-122.38811 37.74780, -122.3880...",San Francisco,884363


In [7]:
counties_and_sf_gdf = pd.concat([filtered_county_gdf,sf_gdf])

In [8]:
counties_and_sf_gdf

,NAME,Population,geometry
27,Napa County,139417.0,"POLYGON ((-122.10328 38.51335, -122.08884 38.3..."
735,Santa Cruz County,274255.0,"POLYGON ((-122.01597 37.16566, -121.75760 37.0..."
736,Solano County,446610.0,"POLYGON ((-122.08884 38.38415, -122.10328 38.5..."
741,Contra Costa County,1150215.0,"POLYGON ((-121.91014 37.73046, -121.96077 37.7..."
866,Santa Cruz County,46511.0,"POLYGON ((-111.16399 31.61869, -111.16418 31.7..."
2002,Marin County,259666.0,"POLYGON ((-122.49073 38.10975, -122.49128 38.1..."
2311,Sonoma County,499942.0,"POLYGON ((-123.26871 38.80837, -123.13625 38.8..."
1,San Francisco,884363.0,"MULTIPOLYGON (((-122.38811 37.74780, -122.3880..."


In [16]:
result_gdf = sum_per_region(ccm_facility_data_gpd,
                   counties_and_sf_gdf,
                   groupby_columns='NAME',
                   region_id_column='NAME',
                   population_columns={'People': 'Population'})
result_df = result_gdf[['NAME',
                        'Total Staffed ICU Beds',
                        'Total Staffed Beds',
                        'Total Licensed Beds',
                        'Bed Occupancy Rate',
                        'ICU Bed Occupancy Rate',
                        'Estimated Beds Available (Conventional) [Per 1000 People]',
                        'Estimated Beds Available (Conventional) [Per 1000 People]',
                        'Estimated Beds Available (Contingency) [Per 1000 People]',
                        'Estimated Beds Available (Contingency) [Per 1000 People]',
                        'Estimated Beds Available (Crisis) [Per 1000 People]',
                        'Estimated Beds Available (Crisis) [Per 1000 People]']]
result_df

,NAME,Total Staffed ICU Beds,Total Staffed Beds,Total Licensed Beds,Bed Occupancy Rate,ICU Bed Occupancy Rate,Estimated Beds Available (Conventional) [Per 1000 People],Estimated Beds Available (Conventional) [Per 1000 People],Estimated Beds Available (Contingency) [Per 1000 People],Estimated Beds Available (Contingency) [Per 1000 People],Estimated Beds Available (Crisis) [Per 1000 People],Estimated Beds Available (Crisis) [Per 1000 People]
0,Contra Costa County,163.0,1636.0,1697.0,0.55,0.60,0.056,0.056,0.182,0.182,0.660,0.660
1,Marin County,66.0,481.0,481.0,0.51,0.66,0.089,0.089,0.281,0.281,0.863,0.863
2,Napa County,48.0,318.0,408.0,0.51,0.32,0.230,0.230,0.746,0.746,1.528,1.528
3,San Francisco,343.0,2974.0,3332.0,0.55,0.59,0.161,0.161,0.513,0.513,1.819,1.819
4,Santa Cruz County,22.0,379.0,399.0,0.55,0.71,0.026,0.026,0.077,0.077,0.638,0.638
5,Santa Cruz County,22.0,379.0,399.0,0.55,0.71,0.151,0.151,0.452,0.452,3.763,3.763
6,Solano County,92.0,1067.0,1117.0,0.40,0.44,0.114,0.114,0.367,0.367,1.516,1.516
7,Sonoma County,82.0,824.0,827.0,0.58,0.56,0.072,0.072,0.232,0.232,0.690,0.690


In [18]:
result_df.to_csv('../data/sf-vs-bay-area.csv')